- Date: 01/01/2024
- Author: Carlos Javier Navarro
- Project: EarthCul

### Code to calculate count in points, or lines or polygons

Script used to download the layers of interest in [Open Street Map](https://www.openstreetmap.org/#map=6/40.01/-2.49)

1) Import libraries

In [1]:
#%pip install overpass
import geopandas as gpd
import overpass
import geojson
import os
from pathlib import Path

2) Download variables using key words of the variables of interest and using the nwr geometry type that includes nodes, ways and relations

In [2]:

base_path = os.getcwd()

#Lists of parks of interest
#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide
park = 'Aiguestortes'

# Define the output directory for the results
OUTPUT_DIR = Path(os.path.join(base_path, f"EarthCul OSM/{park}"))

# Check if the output directory exists; if not, create it
if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir(parents=True)

## Load the bounding box of each park
gdf = gpd.read_file(os.path.join(base_path, f"../BoundingBox/{park}.shp"))
# Convert the GeoDataFrame to EPSG:4326
gdf = gdf.to_crs('EPSG:4326')

aoi = gdf.geometry.bounds.iloc[0].to_dict()
bbox = f"{aoi['miny']},{aoi['minx']},{aoi['maxy']},{aoi['maxx']}"
bboxname = f'{park}'

def save_as_geojson(feature, tag, bbox, feat_type='nwr'):
    api = overpass.API()

    try:
        res = api.get(
            f"""
            ({feat_type}["{feature}"="{tag}"]({bbox});
            );
            """,
            verbosity='geom'
        )
    except:
        return None

    # dump as file, if you want to save it in file
    with open(OUTPUT_DIR.joinpath(f"{feature}_{tag}.geojson"), mode="w") as f:
        geojson.dump(res, f)


# Lista de características y etiquetas a descargar
features_to_download = [
('amenity','restaurant'),
('amenity','bar'),
('amenity','cafe'),
('natural','coastline'),
('natural','beach'),
('amenity','bicycle_rental'),
('route','mtb'),
('route','bicycle'),
# ('building',None),
('highway','bus_stop'),
('tourism','camp_site'),
('tourism','caravan_site'),
('tourism','camp_pitch'),
('sport','climbing'),
('amenity','community_centre'),
('leisure','pitch'),
('amenity','drinking_water'),
('amenity','fountain'),
# ('geological',None),
('geological','moraine'),
('natural','moraine'),
('natural','glacier'),
('leisure','park'),
('tourism','picnic_site'),
('route','hiking'),
('historic','monastery'),
('amenity','monastery'),
('water','lake'),
('water','lagoon'),
('water','oxbow'),
('historic','monument'),
('historic','memorial'),
('amenity','parking'),
('amenity','bicycle_parking'),
('amenity','motorcycle_parking'),
('building','parking'),
('parking','multi-storey'),
('tourism','hostel'),
('tourism','hotel'),
('tourism','motel'),
('tourism','information'),
('railway','rail'),
('landuse','recreation_ground'),
('leisure','playground'),
('shelter_type','basic_hut'),
('amenity','shelter_type'),
('tourism','alpine_hut'),
('building','cathedral'),
('building','chapel'),
('building','church'),
('building','kingdom_hall'),
('building','monastery'),
('building','mosque'),
('building','presbytery'),
('building','religious'),
('building','synagogue'),
('building','temple'),
('amenity','place_of_worship'),
('leisure','resort'),
('waterway','river'),
('waterway','riverbank'),
('water','river'),
('water','stream'),
('waterway','stream'),
('highway','motorway'),
('highway','motorway_link'),
('highway','trunk'),
('highway','trunk_link'),
('highway','primary'),
('highway','secondary'),
('highway','tertiary'),
('highway','unclassified'),
('highway','trailhead'),
('natural','peak'),
('natural','ridge'),
('natural','arete'),
('natural','cliff'),
('tourism','viewpoint'),
('route','ski'),
('route','piste'),
('amenity','ski_rental'),
('place','town'),
('place','hamlet'),
('place','isolated_dwelling'),
('place','village'),
('place','neighbourhood'),
('place','locality'),
('landuse','residential'),
('amenity','social_facility'),
('place','square'),
('highway','living_street'),
('highway','service'),
('highway','residential'),
('tourism','attraction'),
('tourism','artwork'),
('tourism','museum'),
('highway','track'),
('residential','urban'),
('place','city'),
('landuse','orchard'),
('landuse','farmland'),
('landuse','farmyard'),
('landuse','forest'),
('natural','wood'),
('landuse','meadow'),
('natural','grassland'),
('landuse','grass'),
('natural','wetland'),
('natural','mud'),
('landuse','salt_pond'),
('natural','volcano'),
('geological','volcanic_caldera_rim'),
('geological','volcanic_lava_field'),
('geological','volcanic_vent'),
('natural','water'),
('landuse','reservoir'),
('landuse','basin'),
('water','reservoir'),
# ('wifi', None),
('internet_access','wlan'),
('geological','paleontological_site'),
('tourism','wilderness_hut'), 
('route', 'piste'),
('aerialway', 'chair_lift'),
('aerialway', 'drag_lift'),
('landuse', 'winter_sports'),
('sport', 'skiing')
]

for feature, tag in features_to_download:
    save_as_geojson(feature, tag, bbox)